# RWKV v5-slim / embedding init-range 1e-01 / 4k

- 6 layers
- 4096 embedding size

Going through the modified memory training for v5 models, across various initial embedding model weights

**Note:** This project assumes you have the rwkv-infctx conda env setup

# Basic Setup

In [1]:
# First lets setup the various directories, and init the model
!mkdir -p ../../../../model/
!mkdir -p ../../../../datapath/
!mkdir -p ../../../../checkpoint/

In [2]:
# Additional dependencies for eval stuff
!pip install -q aiocsv aiofiles


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python3.11 -m pip install --upgrade pip


In [3]:
DEEPSPEED_STRAT="deepspeed_stage_1"
GPU_DEVICES="auto"
ENABLE_WANDB=True

RWKV_WAVENET_LAYERS=1

EMBED_SCALE=0.1
EMBED_SCALE_LABEL=str(EMBED_SCALE).replace(".", "_")

LAYER_COUNT=6
EMBED_DIM=4096

WANDB_PREFIX=f"v5-L{LAYER_COUNT}-D{EMBED_DIM}-E{EMBED_SCALE}"
FILENAME_PREFIX=f"v5-L{LAYER_COUNT}-D{EMBED_DIM}-E{EMBED_SCALE_LABEL}"

print("DEEPSPEED_STRAT:", DEEPSPEED_STRAT)
print("ENABLE_WANDB:", ENABLE_WANDB)
print("GPU_DEVICES:", GPU_DEVICES)

if ENABLE_WANDB:
    WANDB_MODE="online"
else:
    WANDB_MODE="disabled"

# Computing the notebook, and various paths
import os
NOTEBOOK_DIR=os.path.dirname(os.path.abspath("__file__"))
PROJECT_DIR=os.path.abspath(os.path.join(NOTEBOOK_DIR, "../../../../"))
TRAINER_DIR=os.path.abspath(os.path.join(PROJECT_DIR, "./RWKV-v5/"))
INFERENCE_DIR=os.path.abspath(os.path.join(PROJECT_DIR, "./RWKV-v5/"))

print("NOTEBOOK_DIR:", NOTEBOOK_DIR)
print("INFERENCE_DIR:", INFERENCE_DIR)
print("TRAINER_DIR:", TRAINER_DIR)
print("PROJECT_DIR:", PROJECT_DIR)

DEEPSPEED_STRAT: deepspeed_stage_1
ENABLE_WANDB: True
GPU_DEVICES: auto
NOTEBOOK_DIR: /root/rwkv-x-playground/notebook/experiment/rwkv-x-exp/v5-slim-memory
INFERENCE_DIR: /root/rwkv-x-playground/RWKV-v5
TRAINER_DIR: /root/rwkv-x-playground/RWKV-v5
PROJECT_DIR: /root/rwkv-x-playground


In [4]:
# Init the model
!cd "{TRAINER_DIR}" && \
    export RWKV_WAVENET_LAYERS="{RWKV_WAVENET_LAYERS}" && \
    python3 ./init_model.py \
        --n_layer 6 --n_embd 4096 \
        --emb-scale "{EMBED_SCALE}" \
        --vocab_size neox --skip-if-exists \
        "../model/L{LAYER_COUNT}-D{EMBED_DIM}-E{EMBED_SCALE_LABEL}-neox-v5base-init.pth"

Setting ds_accelerator to cuda (auto detect)
[RWKV.model] Running RWKV model using 'torch-jit' with torch '2.0.1+cu118'
---- Initializing model ----
No of layers: 6
Embedding size: 4096
Output model path: ../model/L6-D4096-E0_1-neox-v5base-init.pth
Vocab size: 50277
Emb scale: 0.1
Note: this process takes a significant time (and ram) for large models
---- ----- ----
50277 4096  -0.1 emb.weight
4096  4096  1.0  blocks.0.att.receptance.weight
4096  4096  1.0  blocks.0.att.key.weight
4096  4096  1.0  blocks.0.att.value.weight
4096  4096  0    blocks.0.att.output.weight
16384 4096  1.0  blocks.0.ffn.key.weight
4096  4096  0    blocks.0.ffn.receptance.weight
4096  16384 0    blocks.0.ffn.value.weight
4096  4096  1.0  blocks.1.att.receptance.weight
4096  4096  1.0  blocks.1.att.key.weight
4096  4096  1.0  blocks.1.att.value.weight
4096  4096  0    blocks.1.att.output.weight
16384 4096  1.0  blocks.1.ffn.key.weight
4096  4096  0    blocks.1.ffn.receptance.weight
4096  16384 0    blocks.1.ffn.

## Enwiki Stage 1 : Foundation 4k model training

In [5]:
# Lets preload the requried dataset 
!cd "{TRAINER_DIR}" && \
    python3 preload_datapath.py "{NOTEBOOK_DIR}/v5base-enwiki-4k.yaml"






Extracting data files: 100%|█████████████████████| 1/1 [00:00<00:00, 376.14it/s]
Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/teven___parquet/teven--enwiki_100k-1359e81b212c2dd6/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7. Subsequent calls will reuse this data.
100%|█████████████████████████████████████████████| 1/1 [00:00<00:00, 47.73it/s]


In [6]:
# Start the foundation model training
!cd "{TRAINER_DIR}" && \
    export RWKV_WAVENET_LAYERS="{RWKV_WAVENET_LAYERS}" && \
    export WANDB_MODE="{WANDB_MODE}" && \
    python lightning_trainer.py fit \
        -c "{NOTEBOOK_DIR}/v5base-enwiki-4k.yaml" \
        --trainer.logger.init_args.name="{WANDB_PREFIX} - Enwiki-4k Foundation (train-ctx=4k, {DEEPSPEED_STRAT})" \
        --trainer.strategy="{DEEPSPEED_STRAT}" \
        --trainer.devices="{GPU_DEVICES}" \
        --trainer.callbacks.init_args.dirpath="../checkpoint/{FILENAME_PREFIX}-enwiki-4k/" \
        --model.load_model="../model/L{LAYER_COUNT}-D{EMBED_DIM}-E{EMBED_SCALE_LABEL}-neox-v5base-init.pth" \
        --model.ctx_len=4096 \
        --model.bptt_learning_range=1

Setting ds_accelerator to cuda (auto detect)
[RWKV.model] Running RWKV model using 'torch-jit' with torch '2.0.1+cu118'
/usr/local/lib/python3.11/dist-packages/lightning/pytorch/cli.py:484: UserWarning: LightningCLI's args parameter is intended to run from within Python like if it were from the command line. To prevent mistakes it is not recommended to provide both args and command line arguments, got: sys.argv[1:]=['fit', '-c', '/root/rwkv-x-playground/notebook/experiment/rwkv-x-exp/v5-slim-memory/v5base-enwiki-4k.yaml', '--trainer.logger.init_args.name=v5-L6-D4096-E0.1 - Enwiki-4k Foundation (train-ctx=4k, deepspeed_stage_1)', '--trainer.strategy=deepspeed_stage_1', '--trainer.devices=auto', '--trainer.callbacks.init_args.dirpath=../checkpoint/v5-L6-D4096-E0_1-enwiki-4k/', '--model.load_model=../model/L6-D4096-E0_1-neox-v5base-init.pth', '--model.ctx_len=4096', '--model.bptt_learning_range=1'], args=['fit', '-c', '/root/rwkv-x-playground/notebook/experiment/rwkv-x-exp/v5-slim-memory/

In [40]:
# Lets export the model from the checkpoint
!cd "{TRAINER_DIR}" && \
    python export_checkpoint.py "../checkpoint/{FILENAME_PREFIX}-enwiki-4k/last.ckpt" "../model/{FILENAME_PREFIX}-enwiki-4k.pth" "bf16"
!cd "{TRAINER_DIR}" && ls -alh "../model/{FILENAME_PREFIX}-enwiki-4k.pth"

Setting ds_accelerator to cuda (auto detect)
Processing zero checkpoint '../checkpoint/v5-L6-D4096-E0_1-enwiki-4k/last.ckpt/checkpoint'
Detected checkpoint of type zero stage ZeroStageEnum.optimizer_states, world_size: 8
Parsing checkpoint created by deepspeed==0.9.3
Reconstructed fp32 state dict with 126 params 1720779520 elements
Saving bf16 state dict to ../model/v5-L6-D4096-E0_1-enwiki-4k.pth
-rw-r--r-- 1 root root 3.3G Aug 19 19:22 ../model/v5-L6-D4096-E0_1-enwiki-4k.pth


In [41]:
# # Lets do a quick dragon prompt validation
!cd "{INFERENCE_DIR}" && \
    export RWKV_WAVENET_LAYERS="{RWKV_WAVENET_LAYERS}" && \
    python3 dragon_test.py "../model/{FILENAME_PREFIX}-enwiki-4k.pth" "cuda fp32"

Setting ds_accelerator to cuda (auto detect)
[RWKV.model] Running RWKV model using 'torch-jit' with torch '2.0.1+cu118'
--- DRAGON PROMPT ---
In a shocking finding, scientist discovered a herd of dragons living in a remote, previously unexplored valley, in Tibet. Even more surprising to the researchers was the fact that the dragons spoke perfect Chinese. He saw how the rice fields provided rice to them, while still other farmers had reached the mountains. He claimed that the great king, however, could not tell them about the mountain's world.

In an early 20th-century visit to Beijing in 1805, Andrew Dana, then the ambassador to the National Academy of Sciences of China, stated that the mountain was the best in the world. He was deeply disturbed by the mountain's water, and urged to continue their journey, and eventually sent the army to the center. The army responded to their efforts by promising to raise the mountain's water, and the mountain was destroyed by fire in late 1819.

By 1

In [42]:
# Lets do a quick memory test
!export RWKV_WAVENET_LAYERS="{RWKV_WAVENET_LAYERS}" && \
        python3 ../memory_script/eval_v5_memory_guided.py "{PROJECT_DIR}/model/{FILENAME_PREFIX}-enwiki-4k.pth"

Setting ds_accelerator to cuda (auto detect)
[RWKV.model] Running RWKV model using 'torch-jit' with torch '2.0.1+cu118'
###
### Model validation start ###
###
## Model validation for 5 tokens : 0.0% similarity, with 0 matched token, and 5 token mismatch
## Model validation for 10 tokens : 10.0% similarity, with 1 matched token, and 9 token mismatch
## Model validation for 15 tokens : 0.0% similarity, with 0 matched token, and 15 token mismatch
## Model validation for 20 tokens : 5.0% similarity, with 1 matched token, and 19 token mismatch
## Model validation for 25 tokens : 4.0% similarity, with 1 matched token, and 24 token mismatch
## Model validation for 30 tokens : 3.3333333333333335% similarity, with 1 matched token, and 29 token mismatch
## Model validation for 35 tokens : 2.857142857142857% similarity, with 1 matched token, and 34 token mismatch
## Model validation for 40 tokens : 0.0% similarity, with 0 matched token, and 40 token mismatch
## Model validation for 45 tokens : 0.

# Enwiki Stage 2 : Basic Instruct Tuning

In [43]:
# Lets preload the requried dataset
!cd "{TRAINER_DIR}" && \
    python3 preload_datapath.py "{NOTEBOOK_DIR}/v5base-enwiki-instruct.yaml"

Found cached dataset parquet (/root/.cache/huggingface/datasets/c-s-ale___parquet/c-s-ale--dolly-15k-instruction-alpaca-format-9dfbb23260d63d9d/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7)
100%|████████████████████████████████████████████| 1/1 [00:00<00:00, 652.40it/s]
Loading cached processed dataset at /root/.cache/huggingface/datasets/c-s-ale___parquet/c-s-ale--dolly-15k-instruction-alpaca-format-9dfbb23260d63d9d/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7/cache-3a81f68e4498c60a_*_of_00064.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/c-s-ale___parquet/c-s-ale--dolly-15k-instruction-alpaca-format-9dfbb23260d63d9d/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7/cache-36c9ee56cc63a264_*_of_00064.arrow
Loading cached split indices for dataset at /root/.cache/huggingface/datasets/c-s-ale___parquet/c-s-ale--dolly-15k-instruction-alpaca-format-9dfbb23260d63d9d/0.0.0/14a00e99c0d15a236

In [44]:
# Start the instruct finetuning
!cd "{TRAINER_DIR}" && \
    export WANDB_MODE="{WANDB_MODE}" && \
    export RWKV_WAVENET_LAYERS="{RWKV_WAVENET_LAYERS}" && \
    python lightning_trainer.py fit \
        -c "{NOTEBOOK_DIR}/v5base-enwiki-instruct.yaml" \
        --trainer.logger.init_args.name="{WANDB_PREFIX} - Enwiki-Instruct (train-ctx=4k, {DEEPSPEED_STRAT})" \
        --trainer.strategy="{DEEPSPEED_STRAT}" \
        --trainer.devices="{GPU_DEVICES}" \
        --trainer.callbacks.init_args.dirpath="../checkpoint/{FILENAME_PREFIX}-enwiki-instruct/" \
        --model.load_model="../model/{FILENAME_PREFIX}-enwiki-4k.pth" \
        --model.ctx_len=4096 \
        --model.bptt_learning_range=1

Setting ds_accelerator to cuda (auto detect)
[RWKV.model] Running RWKV model using 'torch-jit' with torch '2.0.1+cu118'
/usr/local/lib/python3.11/dist-packages/lightning/pytorch/cli.py:484: UserWarning: LightningCLI's args parameter is intended to run from within Python like if it were from the command line. To prevent mistakes it is not recommended to provide both args and command line arguments, got: sys.argv[1:]=['fit', '-c', '/root/rwkv-x-playground/notebook/experiment/rwkv-x-exp/v5-slim-memory/v5base-enwiki-instruct.yaml', '--trainer.logger.init_args.name=v5-L6-D4096-E0.1 - Enwiki-Instruct (train-ctx=4k, deepspeed_stage_1)', '--trainer.strategy=deepspeed_stage_1', '--trainer.devices=auto', '--trainer.callbacks.init_args.dirpath=../checkpoint/v5-L6-D4096-E0_1-enwiki-instruct/', '--model.load_model=../model/v5-L6-D4096-E0_1-enwiki-4k.pth', '--model.ctx_len=4096', '--model.bptt_learning_range=1'], args=['fit', '-c', '/root/rwkv-x-playground/notebook/experiment/rwkv-x-exp/v5-slim-memo

In [45]:
# Lets export the model from the checkpoint
!cd "{TRAINER_DIR}" && \
    python export_checkpoint.py "../checkpoint/{FILENAME_PREFIX}-enwiki-instruct/last.ckpt" "../model/{FILENAME_PREFIX}-enwiki-instruct.pth" "bf16"
!cd "{TRAINER_DIR}" && ls -alh "../model/{FILENAME_PREFIX}-enwiki-instruct.pth"

Setting ds_accelerator to cuda (auto detect)
Processing zero checkpoint '../checkpoint/v5-L6-D4096-E0_1-enwiki-instruct/last.ckpt/checkpoint'
Detected checkpoint of type zero stage ZeroStageEnum.optimizer_states, world_size: 8
Parsing checkpoint created by deepspeed==0.9.3
Reconstructed fp32 state dict with 126 params 1720779520 elements
Saving bf16 state dict to ../model/v5-L6-D4096-E0_1-enwiki-instruct.pth
-rw-r--r-- 1 root root 3.3G Aug 19 20:14 ../model/v5-L6-D4096-E0_1-enwiki-instruct.pth


In [46]:
# # Lets do a quick dragon prompt validation
!cd "{INFERENCE_DIR}" && \
    export RWKV_WAVENET_LAYERS="{RWKV_WAVENET_LAYERS}" && \
    python3 dragon_test.py "../model/{FILENAME_PREFIX}-enwiki-instruct.pth" "cuda fp32"

Setting ds_accelerator to cuda (auto detect)
[RWKV.model] Running RWKV model using 'torch-jit' with torch '2.0.1+cu118'
--- DRAGON PROMPT ---
In a shocking finding, scientist discovered a herd of dragons living in a remote, previously unexplored valley, in Tibet. Even more surprising to the researchers was the fact that the dragons spoke perfect Chinese. One particular interesting aspect was that the artifacts and trade show world cultures were interdependent and dependent on the time of whom they were enjoying.

A number of reasons is that the dragons first appeared on their own and described their significance.

A trip to the forest, along with a number of rare artifacts and flower breeds, including the Chinese on planet Earth, was given to the Chinese and Japanese scholars.# Answer~.nThe Chinese's trail was a Chinese feature that made the Chinese a Chinese thinker.

A glass panel was one of the best ways to get to the Japanese since it is widely believed that Chinese dragons might h

In [47]:
# Lets do a quick memory test
!export RWKV_WAVENET_LAYERS="{RWKV_WAVENET_LAYERS}" && \
        python3 ../memory_script/eval_v5_memory_guided.py "{PROJECT_DIR}/model/{FILENAME_PREFIX}-enwiki-instruct.pth"

Setting ds_accelerator to cuda (auto detect)
[RWKV.model] Running RWKV model using 'torch-jit' with torch '2.0.1+cu118'
###
### Model validation start ###
###
## Model validation for 5 tokens : 40.0% similarity, with 2 matched token, and 3 token mismatch
## Model validation for 10 tokens : 20.0% similarity, with 2 matched token, and 8 token mismatch
## Model validation for 15 tokens : 13.333333333333334% similarity, with 2 matched token, and 13 token mismatch
## Model validation for 20 tokens : 10.0% similarity, with 2 matched token, and 18 token mismatch
## Model validation for 25 tokens : 4.0% similarity, with 1 matched token, and 24 token mismatch
## Model validation for 30 tokens : 3.3333333333333335% similarity, with 1 matched token, and 29 token mismatch
## Model validation for 35 tokens : 2.857142857142857% similarity, with 1 matched token, and 34 token mismatch
## Model validation for 40 tokens : 2.5% similarity, with 1 matched token, and 39 token mismatch
## Model validation f

## Tune 1 : Simple Memory instruct finetuning

- Tune 1: Low ctx size (512), Training with only the input masked. This does very limited memory training, and is used primarily to train the instruction set.

In [48]:
%%script bash

########################################
# Generate the required jsonl dataset
########################################

# Reset the dataset dir
mkdir -p ../dataset
rm -rf ../dataset/*.jsonl

# Generate the various datasets
echo "## Generating word reptition dataset ##"

# We do a strong bias for smaller word count, to teach the concept from scratch
# so that the model can learn the function. 
#
# Note that all document samples, are randomized between the target word count, 
# to half of the target word count.
python ../memory_script/gen_limited_segmented_jsonl.py ../dataset/word-2-count.jsonl  2  5000 &
python ../memory_script/gen_limited_segmented_jsonl.py ../dataset/word-5-count.jsonl  5  5000 &
python ../memory_script/gen_limited_segmented_jsonl.py ../dataset/word-10-count.jsonl 10 2500 &
python ../memory_script/gen_limited_segmented_jsonl.py ../dataset/word-15-count.jsonl 15 2500 &
python ../memory_script/gen_limited_segmented_jsonl.py ../dataset/word-20-count.jsonl 20 2500 &
python ../memory_script/gen_limited_segmented_jsonl.py ../dataset/word-25-count.jsonl 25 2500 &
python ../memory_script/gen_limited_segmented_jsonl.py ../dataset/word-40-count.jsonl 40 2500 &
python ../memory_script/gen_limited_segmented_jsonl.py ../dataset/word-50-count.jsonl 50 2500 &
python ../memory_script/gen_limited_segmented_jsonl.py ../dataset/word-60-count.jsonl 80 2500 &
python ../memory_script/gen_limited_segmented_jsonl.py ../dataset/word-80-count.jsonl 80 2500 &

# With a slight mix of the larger word count
python ../memory_script/gen_limited_segmented_jsonl.py ../dataset/word-100-count.jsonl 100 2500 &
python ../memory_script/gen_limited_segmented_jsonl.py ../dataset/word-200-count.jsonl 200 2500 &

wait
echo "## Done ##"

ls -alh ../dataset/

## Generating word reptition dataset ##
Generated JSONL file with - 10 max words, 2500 samples - at ../dataset/word-10-count.jsonl
Generated JSONL file with - 15 max words, 2500 samples - at ../dataset/word-15-count.jsonl
Generated JSONL file with - 20 max words, 2500 samples - at ../dataset/word-20-count.jsonl
Generated JSONL file with - 2 max words, 5000 samples - at ../dataset/word-2-count.jsonl
Generated JSONL file with - 25 max words, 2500 samples - at ../dataset/word-25-count.jsonl
Generated JSONL file with - 5 max words, 5000 samples - at ../dataset/word-5-count.jsonl
Generated JSONL file with - 40 max words, 2500 samples - at ../dataset/word-40-count.jsonl
Generated JSONL file with - 50 max words, 2500 samples - at ../dataset/word-50-count.jsonl
Generated JSONL file with - 80 max words, 2500 samples - at ../dataset/word-60-count.jsonl
Generated JSONL file with - 80 max words, 2500 samples - at ../dataset/word-80-count.jsonl
Generated JSONL file with - 100 max words, 2500 sample

In [49]:
# Start the finetune model training
!cd "{TRAINER_DIR}" && \
    export WANDB_MODE="{WANDB_MODE}" && \
    export RWKV_WAVENET_LAYERS="{RWKV_WAVENET_LAYERS}" && \
    python lightning_trainer.py fit \
        -c "{NOTEBOOK_DIR}/v5base-mem-instruct.yaml" \
        --trainer.logger.init_args.name="{WANDB_PREFIX} - Mem-Instruct (train-ctx=512, {DEEPSPEED_STRAT})" \
        --trainer.strategy="{DEEPSPEED_STRAT}" \
        --trainer.devices="{GPU_DEVICES}" \
        --trainer.callbacks.init_args.dirpath="../checkpoint/{FILENAME_PREFIX}-mem-instruct/" \
        --model.load_model="../model/{FILENAME_PREFIX}-enwiki-instruct.pth" \
        --model.ctx_len=512 \
        --model.bptt_learning_range=1

Setting ds_accelerator to cuda (auto detect)
[RWKV.model] Running RWKV model using 'torch-jit' with torch '2.0.1+cu118'
/usr/local/lib/python3.11/dist-packages/lightning/pytorch/cli.py:484: UserWarning: LightningCLI's args parameter is intended to run from within Python like if it were from the command line. To prevent mistakes it is not recommended to provide both args and command line arguments, got: sys.argv[1:]=['fit', '-c', '/root/rwkv-x-playground/notebook/experiment/rwkv-x-exp/v5-slim-memory/v5base-mem-instruct.yaml', '--trainer.logger.init_args.name=v5-L6-D4096-E0.1 - Mem-Instruct (train-ctx=512, deepspeed_stage_1)', '--trainer.strategy=deepspeed_stage_1', '--trainer.devices=auto', '--trainer.callbacks.init_args.dirpath=../checkpoint/v5-L6-D4096-E0_1-mem-instruct/', '--model.load_model=../model/v5-L6-D4096-E0_1-enwiki-instruct.pth', '--model.ctx_len=512', '--model.bptt_learning_range=1'], args=['fit', '-c', '/root/rwkv-x-playground/notebook/experiment/rwkv-x-exp/v5-slim-memory/

In [50]:
# Lets export the model from the checkpoint
!cd "{TRAINER_DIR}" && \
    python export_checkpoint.py \
        "../checkpoint/{FILENAME_PREFIX}-mem-instruct/last.ckpt" \
        "../model/{FILENAME_PREFIX}-mem-instruct.pth" "bf16"
!cd "{TRAINER_DIR}" && ls -alh "../model/{FILENAME_PREFIX}-mem-instruct.pth"

Setting ds_accelerator to cuda (auto detect)
Processing zero checkpoint '../checkpoint/v5-L6-D4096-E0_1-mem-instruct/last.ckpt/checkpoint'
Detected checkpoint of type zero stage ZeroStageEnum.optimizer_states, world_size: 8
Parsing checkpoint created by deepspeed==0.9.3
Reconstructed fp32 state dict with 126 params 1720779520 elements
Saving bf16 state dict to ../model/v5-L6-D4096-E0_1-mem-instruct.pth
-rw-r--r-- 1 root root 3.3G Aug 19 20:40 ../model/v5-L6-D4096-E0_1-mem-instruct.pth


In [51]:
# Lets do a quick memory test
!export RWKV_WAVENET_LAYERS="{RWKV_WAVENET_LAYERS}" && \
        python3 ../memory_script/eval_v5_memory_guided.py "{PROJECT_DIR}/model/{FILENAME_PREFIX}-mem-instruct.pth"

Setting ds_accelerator to cuda (auto detect)
[RWKV.model] Running RWKV model using 'torch-jit' with torch '2.0.1+cu118'
###
### Model validation start ###
###
## Model validation for 5 tokens : 100.0% similarity, with 5 matched token, and 0 token mismatch
## Model validation for 10 tokens : 100.0% similarity, with 10 matched token, and 0 token mismatch
## Model validation for 15 tokens : 100.0% similarity, with 15 matched token, and 0 token mismatch
## Model validation for 20 tokens : 100.0% similarity, with 20 matched token, and 0 token mismatch
## Model validation for 25 tokens : 96.0% similarity, with 24 matched token, and 1 token mismatch
## Model validation for 30 tokens : 96.66666666666667% similarity, with 29 matched token, and 1 token mismatch
## Model validation for 35 tokens : 97.14285714285714% similarity, with 34 matched token, and 1 token mismatch
## Model validation for 40 tokens : 95.0% similarity, with 38 matched token, and 2 token mismatch
## Model validation for 45 to

## Tune 2 : Low ctx size (512), memory training

- Tune 2: Low ctx size (512), Training with instruction & input masked. This forces the actual memory training on the output tokens.

In [52]:
%%script bash

########################################
# Generate the required jsonl dataset
########################################

# Reset the dataset dir
mkdir -p ../dataset
rm -rf ../dataset/*.jsonl

# Generate the various datasets
echo "## Generating word reptition dataset ##"

#
# We switch over to fully masked instruct+input, to properly learn the memorization task
#
python ../memory_script/gen_limited_prompt_completion_jsonl.py ../dataset/word-2-count.jsonl  2  5000 &
for i in {5..95..5} 
do
    python ../memory_script/gen_limited_prompt_completion_jsonl.py ../dataset/gen-word-$i-count.jsonl $i 5000 & 
done
python ../memory_script/gen_limited_prompt_completion_jsonl.py ../dataset/word-100-count.jsonl 100 5000 &
python ../memory_script/gen_limited_prompt_completion_jsonl.py ../dataset/word-200-count.jsonl 200 5000 &

#
# We mixin the shuffled word list, so that we ensure all words / tokens are learned
# however this might intrduce an exclusion bias (if seen this word, never repeat it), 
# so we limit the mixture of this data samples
#
python ../memory_script/shuffle_limited_prompt_completion_jsonl.py ../dataset/shuffle-word-10-count.jsonl 10 20 &
python ../memory_script/shuffle_limited_prompt_completion_jsonl.py ../dataset/shuffle-word-15-count.jsonl 15 20 &
python ../memory_script/shuffle_limited_prompt_completion_jsonl.py ../dataset/shuffle-word-25-count.jsonl 25 30 &
python ../memory_script/shuffle_limited_prompt_completion_jsonl.py ../dataset/shuffle-word-50-count.jsonl 50 50 &
python ../memory_script/shuffle_limited_prompt_completion_jsonl.py ../dataset/shuffle-word-75-count.jsonl 75 50 &
python ../memory_script/shuffle_limited_prompt_completion_jsonl.py ../dataset/shuffle-word-100-count.jsonl 100 50 &
python ../memory_script/shuffle_limited_prompt_completion_jsonl.py ../dataset/shuffle-word-200-count.jsonl 200 50 &

wait
echo "## Done ##"

ls -alh ../dataset/

## Generating word reptition dataset ##
Generated a single JSONL file with 3542 samples (20 token repeat) - 15 max words - at ../dataset/shuffle-word-15-count.jsonl
Generated a single JSONL file with 667 samples (50 token repeat) - 200 max words - at ../dataset/shuffle-word-200-count.jsonl
Generated a single JSONL file with 3157 samples (30 token repeat) - 25 max words - at ../dataset/shuffle-word-25-count.jsonl
Generated a single JSONL file with 5201 samples (20 token repeat) - 10 max words - at ../dataset/shuffle-word-10-count.jsonl
Generated a single JSONL file with 1771 samples (50 token repeat) - 75 max words - at ../dataset/shuffle-word-75-count.jsonl
Generated a single JSONL file with 1341 samples (50 token repeat) - 100 max words - at ../dataset/shuffle-word-100-count.jsonl
Generated JSONL file with - 20 max words, 5000 samples - at ../dataset/gen-word-20-count.jsonl
Generated JSONL file with - 2 max words, 5000 samples - at ../dataset/word-2-count.jsonl
Generated JSONL file wi

In [53]:
# Start the finetune model training
!cd "{TRAINER_DIR}" && \
    export WANDB_MODE="{WANDB_MODE}" && \
    export RWKV_WAVENET_LAYERS="{RWKV_WAVENET_LAYERS}" && \
    python lightning_trainer.py fit \
        -c "{NOTEBOOK_DIR}/v5base-mem-template.yaml" \
        --trainer.logger.init_args.name="{WANDB_PREFIX} - Mem-Tune ctx-512 (train-ctx=512, {DEEPSPEED_STRAT})" \
        --trainer.strategy="{DEEPSPEED_STRAT}" \
        --trainer.devices="{GPU_DEVICES}"  \
        --trainer.callbacks.init_args.dirpath="../checkpoint/{FILENAME_PREFIX}-mem-ctx-512/" \
        --model.lr_init=5e-4 \
        --model.lr_final=4e-4 \
        --data.max_token_size=512 \
        --model.ctx_len=512 \
        --model.bptt_learning_range=1 \
        --model.load_model="../model/{FILENAME_PREFIX}-mem-instruct.pth"

Setting ds_accelerator to cuda (auto detect)
[RWKV.model] Running RWKV model using 'torch-jit' with torch '2.0.1+cu118'
/usr/local/lib/python3.11/dist-packages/lightning/pytorch/cli.py:484: UserWarning: LightningCLI's args parameter is intended to run from within Python like if it were from the command line. To prevent mistakes it is not recommended to provide both args and command line arguments, got: sys.argv[1:]=['fit', '-c', '/root/rwkv-x-playground/notebook/experiment/rwkv-x-exp/v5-slim-memory/v5base-mem-template.yaml', '--trainer.logger.init_args.name=v5-L6-D4096-E0.1 - Mem-Tune ctx-512 (train-ctx=512, deepspeed_stage_1)', '--trainer.strategy=deepspeed_stage_1', '--trainer.devices=auto', '--trainer.callbacks.init_args.dirpath=../checkpoint/v5-L6-D4096-E0_1-mem-ctx-512/', '--model.lr_init=5e-4', '--model.lr_final=4e-4', '--data.max_token_size=512', '--model.ctx_len=512', '--model.bptt_learning_range=1', '--model.load_model=../model/v5-L6-D4096-E0_1-mem-instruct.pth'], args=['fit',

In [54]:
# Lets export the model from the checkpoint
!cd "{TRAINER_DIR}" && \
    python export_checkpoint.py \
        "../checkpoint/{FILENAME_PREFIX}-mem-ctx-512/last.ckpt" \
        "../model/{FILENAME_PREFIX}-mem-ctx-512.pth" "bf16"
!cd "{TRAINER_DIR}" && ls -alh "../model/{FILENAME_PREFIX}-mem-ctx-512.pth"

Setting ds_accelerator to cuda (auto detect)
Processing zero checkpoint '../checkpoint/v5-L6-D4096-E0_1-mem-ctx-512/last.ckpt/checkpoint'
Detected checkpoint of type zero stage ZeroStageEnum.optimizer_states, world_size: 8
Parsing checkpoint created by deepspeed==0.9.3
Reconstructed fp32 state dict with 126 params 1720779520 elements
Saving bf16 state dict to ../model/v5-L6-D4096-E0_1-mem-ctx-512.pth
-rw-r--r-- 1 root root 3.3G Aug 19 21:55 ../model/v5-L6-D4096-E0_1-mem-ctx-512.pth


In [55]:
# Lets do a quick memory test
!export RWKV_WAVENET_LAYERS="{RWKV_WAVENET_LAYERS}" && \
        python3 ../memory_script/eval_v5_memory_guided.py "{PROJECT_DIR}/model/{FILENAME_PREFIX}-mem-ctx-512.pth"

Setting ds_accelerator to cuda (auto detect)
[RWKV.model] Running RWKV model using 'torch-jit' with torch '2.0.1+cu118'
###
### Model validation start ###
###
## Model validation for 5 tokens : 100.0% similarity, with 5 matched token, and 0 token mismatch
## Model validation for 10 tokens : 100.0% similarity, with 10 matched token, and 0 token mismatch
## Model validation for 15 tokens : 100.0% similarity, with 15 matched token, and 0 token mismatch
## Model validation for 20 tokens : 100.0% similarity, with 20 matched token, and 0 token mismatch
## Model validation for 25 tokens : 100.0% similarity, with 25 matched token, and 0 token mismatch
## Model validation for 30 tokens : 100.0% similarity, with 30 matched token, and 0 token mismatch
## Model validation for 35 tokens : 100.0% similarity, with 35 matched token, and 0 token mismatch
## Model validation for 40 tokens : 100.0% similarity, with 40 matched token, and 0 token mismatch
## Model validation for 45 tokens : 100.0% similari

## Tune 3 : Low ctx size (1024), memory training

- Tune 3: Low ctx size (1024), Scaling up !

In [56]:
%%script bash

########################################
# Generate the required jsonl dataset
########################################

# Reset the dataset dir
mkdir -p ../dataset
rm -rf ../dataset/*.jsonl

# Generate the various datasets
echo "## Generating word reptition dataset ##"

#
# We reduce the training set for lower word count - and shift the focus upwards
#
python ../memory_script/gen_limited_prompt_completion_jsonl.py ../dataset/word-2-count.jsonl 2 400 &
for i in {5..45..5} 
do
    python ../memory_script/gen_limited_prompt_completion_jsonl.py ../dataset/gen-word-$i-count.jsonl $i 400 & 
    python ../memory_script/shuffle_limited_prompt_completion_jsonl.py ../dataset/shuffle-word-$i-count.jsonl $i 10 & 
done

#
# Ramping up the 50+ - 510 words dataset
# 
for i in {50..550..5} 
do
    python ../memory_script/gen_limited_prompt_completion_jsonl.py ../dataset/gen-word-$i-count.jsonl $i 800 & 
    python ../memory_script/shuffle_limited_prompt_completion_jsonl.py ../dataset/shuffle-word-$i-count.jsonl $i 20 & 
done

wait
echo "## Done ##"

ls -alh ../dataset/

## Generating word reptition dataset ##
Generated JSONL file with - 15 max words, 400 samples - at ../dataset/gen-word-15-count.jsonl
Generated JSONL file with - 25 max words, 400 samples - at ../dataset/gen-word-25-count.jsonl
Generated JSONL file with - 10 max words, 400 samples - at ../dataset/gen-word-10-count.jsonl
Generated JSONL file with - 2 max words, 400 samples - at ../dataset/word-2-count.jsonl
Generated JSONL file with - 5 max words, 400 samples - at ../dataset/gen-word-5-count.jsonl
Generated JSONL file with - 30 max words, 400 samples - at ../dataset/gen-word-30-count.jsonl
Generated a single JSONL file with 587 samples (10 token repeat) - 45 max words - at ../dataset/shuffle-word-45-count.jsonl
Generated JSONL file with - 40 max words, 400 samples - at ../dataset/gen-word-40-count.jsonl
Generated JSONL file with - 20 max words, 400 samples - at ../dataset/gen-word-20-count.jsonl
Generated JSONL file with - 45 max words, 400 samples - at ../dataset/gen-word-45-count.json

In [57]:
# Start the finetune model training
!cd "{TRAINER_DIR}" && \
    export WANDB_MODE="{WANDB_MODE}" && \
    export RWKV_WAVENET_LAYERS="{RWKV_WAVENET_LAYERS}" && \
    python lightning_trainer.py fit \
        -c "{NOTEBOOK_DIR}/v5base-mem-template.yaml" \
        --trainer.logger.init_args.name="{WANDB_PREFIX} - Mem-Tune ctx-1k (train-ctx=1k, {DEEPSPEED_STRAT})" \
        --trainer.strategy="{DEEPSPEED_STRAT}" \
        --trainer.devices="{GPU_DEVICES}"  \
        --trainer.callbacks.init_args.dirpath="../checkpoint/{FILENAME_PREFIX}-mem-ctx-1k/" \
        --model.lr_init=4e-4 \
        --model.lr_final=2e-4 \
        --data.max_token_size=1024 \
        --model.ctx_len=1024 \
        --model.bptt_learning_range=1 \
        --model.load_model="../model/{FILENAME_PREFIX}-mem-ctx-512.pth"

Setting ds_accelerator to cuda (auto detect)
[RWKV.model] Running RWKV model using 'torch-jit' with torch '2.0.1+cu118'
/usr/local/lib/python3.11/dist-packages/lightning/pytorch/cli.py:484: UserWarning: LightningCLI's args parameter is intended to run from within Python like if it were from the command line. To prevent mistakes it is not recommended to provide both args and command line arguments, got: sys.argv[1:]=['fit', '-c', '/root/rwkv-x-playground/notebook/experiment/rwkv-x-exp/v5-slim-memory/v5base-mem-template.yaml', '--trainer.logger.init_args.name=v5-L6-D4096-E0.1 - Mem-Tune ctx-1k (train-ctx=1k, deepspeed_stage_1)', '--trainer.strategy=deepspeed_stage_1', '--trainer.devices=auto', '--trainer.callbacks.init_args.dirpath=../checkpoint/v5-L6-D4096-E0_1-mem-ctx-1k/', '--model.lr_init=4e-4', '--model.lr_final=2e-4', '--data.max_token_size=1024', '--model.ctx_len=1024', '--model.bptt_learning_range=1', '--model.load_model=../model/v5-L6-D4096-E0_1-mem-ctx-512.pth'], args=['fit', '

In [58]:
# Lets export the model from the checkpoint
!cd "{TRAINER_DIR}" && \
    python export_checkpoint.py \
        "../checkpoint/{FILENAME_PREFIX}-mem-ctx-1k/last.ckpt" \
        "../model/{FILENAME_PREFIX}-mem-ctx-1k.pth" "bf16"
!cd "{TRAINER_DIR}" && ls -alh "../model/{FILENAME_PREFIX}-mem-ctx-1k.pth"

Setting ds_accelerator to cuda (auto detect)
Processing zero checkpoint '../checkpoint/v5-L6-D4096-E0_1-mem-ctx-1k/last.ckpt/checkpoint'
Detected checkpoint of type zero stage ZeroStageEnum.optimizer_states, world_size: 8
Parsing checkpoint created by deepspeed==0.9.3
Reconstructed fp32 state dict with 126 params 1720779520 elements
Saving bf16 state dict to ../model/v5-L6-D4096-E0_1-mem-ctx-1k.pth
-rw-r--r-- 1 root root 3.3G Aug 19 23:33 ../model/v5-L6-D4096-E0_1-mem-ctx-1k.pth


In [59]:
# Lets do a quick memory test
!export RWKV_WAVENET_LAYERS="{RWKV_WAVENET_LAYERS}" && \
        python3 ../memory_script/eval_v5_memory_guided.py "{PROJECT_DIR}/model/{FILENAME_PREFIX}-mem-ctx-1k.pth"

Setting ds_accelerator to cuda (auto detect)
[RWKV.model] Running RWKV model using 'torch-jit' with torch '2.0.1+cu118'
###
### Model validation start ###
###
## Model validation for 5 tokens : 100.0% similarity, with 5 matched token, and 0 token mismatch
## Model validation for 10 tokens : 100.0% similarity, with 10 matched token, and 0 token mismatch
## Model validation for 15 tokens : 100.0% similarity, with 15 matched token, and 0 token mismatch
## Model validation for 20 tokens : 100.0% similarity, with 20 matched token, and 0 token mismatch
## Model validation for 25 tokens : 100.0% similarity, with 25 matched token, and 0 token mismatch
## Model validation for 30 tokens : 100.0% similarity, with 30 matched token, and 0 token mismatch
## Model validation for 35 tokens : 100.0% similarity, with 35 matched token, and 0 token mismatch
## Model validation for 40 tokens : 100.0% similarity, with 40 matched token, and 0 token mismatch
## Model validation for 45 tokens : 100.0% similari

## Tune 4 : Low ctx size (2048), memory training

- Tune 4: Low ctx size (2048), Scaling up !

In [60]:
%%script bash

########################################
# Generate the required jsonl dataset
########################################

# Reset the dataset dir
mkdir -p ../dataset
rm -rf ../dataset/*.jsonl

# Generate the various datasets
echo "## Generating word reptition dataset ##"

#
# We reduce the training set for lower word count - and shift the focus upwards
#
python ../memory_script/gen_limited_prompt_completion_jsonl.py ../dataset/word-2-count.jsonl 2 100 &
for i in {5..100..5} 
do
    python ../memory_script/gen_limited_prompt_completion_jsonl.py ../dataset/gen-word-$i-count.jsonl $i 100 & 
    python ../memory_script/shuffle_limited_prompt_completion_jsonl.py ../dataset/shuffle-word-$i-count.jsonl $i 1 & 
done

#
# Ramping up the 105+ - 1050 words dataset
# 
for i in {105..2000..5} 
do
    python ../memory_script/gen_limited_prompt_completion_jsonl.py ../dataset/gen-word-$i-count.jsonl $i 200 & 
    python ../memory_script/shuffle_limited_prompt_completion_jsonl.py ../dataset/shuffle-word-$i-count.jsonl $i 20 & 
done

wait
echo "## Done ##"

ls -alh ../dataset/

## Generating word reptition dataset ##
Generated JSONL file with - 2 max words, 100 samples - at ../dataset/word-2-count.jsonl
Generated JSONL file with - 20 max words, 100 samples - at ../dataset/gen-word-20-count.jsonl
Generated JSONL file with - 25 max words, 100 samples - at ../dataset/gen-word-25-count.jsonl
Generated a single JSONL file with 262 samples (1 token repeat) - 10 max words - at ../dataset/shuffle-word-10-count.jsonl
Generated JSONL file with - 55 max words, 100 samples - at ../dataset/gen-word-55-count.jsonl
Generated a single JSONL file with 49 samples (1 token repeat) - 55 max words - at ../dataset/shuffle-word-55-count.jsonl
Generated JSONL file with - 5 max words, 100 samples - at ../dataset/gen-word-5-count.jsonl
Generated JSONL file with - 45 max words, 100 samples - at ../dataset/gen-word-45-count.jsonl
Generated a single JSONL file with 91 samples (1 token repeat) - 30 max words - at ../dataset/shuffle-word-30-count.jsonl
Generated a single JSONL file with 54

In [61]:
# Start the finetune model training
!cd "{TRAINER_DIR}" && \
    export WANDB_MODE="{WANDB_MODE}" && \
    export RWKV_WAVENET_LAYERS="{RWKV_WAVENET_LAYERS}" && \
    python lightning_trainer.py fit \
        -c "{NOTEBOOK_DIR}/v5base-mem-template.yaml" \
        --trainer.logger.init_args.name="{WANDB_PREFIX} - Mem-Tune ctx-2k (train-ctx=2k, {DEEPSPEED_STRAT})" \
        --trainer.strategy="{DEEPSPEED_STRAT}" \
        --trainer.devices="{GPU_DEVICES}"  \
        --trainer.callbacks.init_args.dirpath="../checkpoint/{FILENAME_PREFIX}-mem-ctx-2k/" \
        --model.lr_init=3e-4 \
        --model.lr_final=1e-4 \
        --data.max_token_size=2048 \
        --model.ctx_len=2048 \
        --model.bptt_learning_range=1 \
        --model.load_model="../model/{FILENAME_PREFIX}-mem-ctx-1k.pth"

Setting ds_accelerator to cuda (auto detect)
[RWKV.model] Running RWKV model using 'torch-jit' with torch '2.0.1+cu118'
/usr/local/lib/python3.11/dist-packages/lightning/pytorch/cli.py:484: UserWarning: LightningCLI's args parameter is intended to run from within Python like if it were from the command line. To prevent mistakes it is not recommended to provide both args and command line arguments, got: sys.argv[1:]=['fit', '-c', '/root/rwkv-x-playground/notebook/experiment/rwkv-x-exp/v5-slim-memory/v5base-mem-template.yaml', '--trainer.logger.init_args.name=v5-L6-D4096-E0.1 - Mem-Tune ctx-2k (train-ctx=2k, deepspeed_stage_1)', '--trainer.strategy=deepspeed_stage_1', '--trainer.devices=auto', '--trainer.callbacks.init_args.dirpath=../checkpoint/v5-L6-D4096-E0_1-mem-ctx-2k/', '--model.lr_init=3e-4', '--model.lr_final=1e-4', '--data.max_token_size=2048', '--model.ctx_len=2048', '--model.bptt_learning_range=1', '--model.load_model=../model/v5-L6-D4096-E0_1-mem-ctx-1k.pth'], args=['fit', '-

In [62]:
# Lets export the model from the checkpoint
!cd "{TRAINER_DIR}" && \
    python export_checkpoint.py \
        "../checkpoint/{FILENAME_PREFIX}-mem-ctx-2k/last.ckpt" \
        "../model/{FILENAME_PREFIX}-mem-ctx-2k.pth" "bf16"
!cd "{TRAINER_DIR}" && ls -alh "../model/{FILENAME_PREFIX}-mem-ctx-2k.pth"

Setting ds_accelerator to cuda (auto detect)
Processing zero checkpoint '../checkpoint/v5-L6-D4096-E0_1-mem-ctx-2k/last.ckpt/checkpoint'
Detected checkpoint of type zero stage ZeroStageEnum.optimizer_states, world_size: 8
Parsing checkpoint created by deepspeed==0.9.3
Reconstructed fp32 state dict with 126 params 1720779520 elements
Saving bf16 state dict to ../model/v5-L6-D4096-E0_1-mem-ctx-2k.pth
-rw-r--r-- 1 root root 3.3G Aug 20 01:06 ../model/v5-L6-D4096-E0_1-mem-ctx-2k.pth


In [63]:
# Lets do a quick memory test
!export RWKV_WAVENET_LAYERS="{RWKV_WAVENET_LAYERS}" && \
        python3 ../memory_script/eval_v5_memory_guided.py "{PROJECT_DIR}/model/{FILENAME_PREFIX}-mem-ctx-2k.pth"

Setting ds_accelerator to cuda (auto detect)
[RWKV.model] Running RWKV model using 'torch-jit' with torch '2.0.1+cu118'
###
### Model validation start ###
###
## Model validation for 5 tokens : 100.0% similarity, with 5 matched token, and 0 token mismatch
## Model validation for 10 tokens : 100.0% similarity, with 10 matched token, and 0 token mismatch
## Model validation for 15 tokens : 100.0% similarity, with 15 matched token, and 0 token mismatch
## Model validation for 20 tokens : 100.0% similarity, with 20 matched token, and 0 token mismatch
## Model validation for 25 tokens : 100.0% similarity, with 25 matched token, and 0 token mismatch
## Model validation for 30 tokens : 100.0% similarity, with 30 matched token, and 0 token mismatch
## Model validation for 35 tokens : 100.0% similarity, with 35 matched token, and 0 token mismatch
## Model validation for 40 tokens : 100.0% similarity, with 40 matched token, and 0 token mismatch
## Model validation for 45 tokens : 100.0% similari

## Tune 5 : Ramping up the ctx size (4096), memory training

- Tune 5: Mid ctx size (4096), Scaling up!

In [64]:
%%script bash

########################################
# Generate the required jsonl dataset
########################################

# Reset the dataset dir
mkdir -p ../dataset
rm -rf ../dataset/*.jsonl

# Generate the various datasets
echo "## Generating word reptition dataset ##"

#
# We reduce the training set for < 50 words - and shift the focus upwards
# (aka 50-100 token * 2 : ~100 - 250 token ctx len)
#
python ../memory_script/gen_limited_prompt_completion_jsonl.py ../dataset/word-2-count.jsonl 2 100 &
for i in {5..500..5} 
do
    python ../memory_script/gen_limited_prompt_completion_jsonl.py ../dataset/gen-word-$i-count.jsonl $i 100 & 
    python ../memory_script/shuffle_limited_prompt_completion_jsonl.py ../dataset/shuffle-word-$i-count.jsonl $i 1 & 
done

#
# Ramping up the 50+ - 2100 words dataset
# 
for i in {505..4000..5} 
do
    python ../memory_script/gen_limited_prompt_completion_jsonl.py ../dataset/gen-word-$i-count.jsonl $i 200 & 
    python ../memory_script/shuffle_limited_prompt_completion_jsonl.py ../dataset/shuffle-word-$i-count.jsonl $i 20 & 
done

wait
echo "## Done ##"

ls -alh ../dataset/

## Generating word reptition dataset ##
Generated JSONL file with - 10 max words, 100 samples - at ../dataset/gen-word-10-count.jsonl
Generated a single JSONL file with 63 samples (1 token repeat) - 40 max words - at ../dataset/shuffle-word-40-count.jsonl
Generated JSONL file with - 5 max words, 100 samples - at ../dataset/gen-word-5-count.jsonl
Generated a single JSONL file with 177 samples (1 token repeat) - 15 max words - at ../dataset/shuffle-word-15-count.jsonl
Generated JSONL file with - 15 max words, 100 samples - at ../dataset/gen-word-15-count.jsonl
Generated a single JSONL file with 266 samples (1 token repeat) - 10 max words - at ../dataset/shuffle-word-10-count.jsonl
Generated JSONL file with - 2 max words, 100 samples - at ../dataset/word-2-count.jsonl
Generated JSONL file with - 65 max words, 100 samples - at ../dataset/gen-word-65-count.jsonl
Generated a single JSONL file with 91 samples (1 token repeat) - 30 max words - at ../dataset/shuffle-word-30-count.jsonl
Generate

In [65]:
# Start the finetune model training
!cd "{TRAINER_DIR}" && \
    export WANDB_MODE="{WANDB_MODE}" && \
    export RWKV_WAVENET_LAYERS="{RWKV_WAVENET_LAYERS}" && \
    python lightning_trainer.py fit \
        -c "{NOTEBOOK_DIR}/v5base-mem-template.yaml" \
        --trainer.logger.init_args.name="{WANDB_PREFIX} - Mem-Tune ctx-4k (train-ctx=4k, {DEEPSPEED_STRAT})" \
        --trainer.strategy="{DEEPSPEED_STRAT}" \
        --trainer.devices="{GPU_DEVICES}"  \
        --trainer.callbacks.init_args.dirpath="../checkpoint/{FILENAME_PREFIX}-mem-ctx-4k/" \
        --model.lr_init=3e-4 \
        --model.lr_final=1e-4 \
        --data.max_token_size=4096 \
        --model.ctx_len=4096 \
        --model.bptt_learning_range=1 \
        --model.load_model="../model/{FILENAME_PREFIX}-mem-ctx-1k.pth"

Setting ds_accelerator to cuda (auto detect)
[RWKV.model] Running RWKV model using 'torch-jit' with torch '2.0.1+cu118'
/usr/local/lib/python3.11/dist-packages/lightning/pytorch/cli.py:484: UserWarning: LightningCLI's args parameter is intended to run from within Python like if it were from the command line. To prevent mistakes it is not recommended to provide both args and command line arguments, got: sys.argv[1:]=['fit', '-c', '/root/rwkv-x-playground/notebook/experiment/rwkv-x-exp/v5-slim-memory/v5base-mem-template.yaml', '--trainer.logger.init_args.name=v5-L6-D4096-E0.1 - Mem-Tune ctx-4k (train-ctx=4k, deepspeed_stage_1)', '--trainer.strategy=deepspeed_stage_1', '--trainer.devices=auto', '--trainer.callbacks.init_args.dirpath=../checkpoint/v5-L6-D4096-E0_1-mem-ctx-4k/', '--model.lr_init=3e-4', '--model.lr_final=1e-4', '--data.max_token_size=4096', '--model.ctx_len=4096', '--model.bptt_learning_range=1', '--model.load_model=../model/v5-L6-D4096-E0_1-mem-ctx-1k.pth'], args=['fit', '-

In [66]:
# Lets export the model from the checkpoint
!cd "{TRAINER_DIR}" && \
    python export_checkpoint.py \
        "../checkpoint/{FILENAME_PREFIX}-mem-ctx-4k/last.ckpt" \
        "../model/{FILENAME_PREFIX}-mem-ctx-4k.pth" "bf16"
!cd "{TRAINER_DIR}" && ls -alh "../model/{FILENAME_PREFIX}-mem-ctx-4k.pth"

Setting ds_accelerator to cuda (auto detect)
Processing zero checkpoint '../checkpoint/v5-L6-D4096-E0_1-mem-ctx-4k/last.ckpt/checkpoint'
Detected checkpoint of type zero stage ZeroStageEnum.optimizer_states, world_size: 8
Parsing checkpoint created by deepspeed==0.9.3
Reconstructed fp32 state dict with 126 params 1720779520 elements
Saving bf16 state dict to ../model/v5-L6-D4096-E0_1-mem-ctx-4k.pth
-rw-r--r-- 1 root root 3.3G Aug 20 05:04 ../model/v5-L6-D4096-E0_1-mem-ctx-4k.pth


In [67]:
# Lets do a quick memory test
!export RWKV_WAVENET_LAYERS="{RWKV_WAVENET_LAYERS}" && \
        python3 ../memory_script/eval_v5_memory_guided.py "{PROJECT_DIR}/model/{FILENAME_PREFIX}-mem-ctx-4k.pth"

Setting ds_accelerator to cuda (auto detect)
[RWKV.model] Running RWKV model using 'torch-jit' with torch '2.0.1+cu118'
###
### Model validation start ###
###
## Model validation for 5 tokens : 100.0% similarity, with 5 matched token, and 0 token mismatch
## Model validation for 10 tokens : 100.0% similarity, with 10 matched token, and 0 token mismatch
## Model validation for 15 tokens : 100.0% similarity, with 15 matched token, and 0 token mismatch
## Model validation for 20 tokens : 100.0% similarity, with 20 matched token, and 0 token mismatch
## Model validation for 25 tokens : 100.0% similarity, with 25 matched token, and 0 token mismatch
## Model validation for 30 tokens : 100.0% similarity, with 30 matched token, and 0 token mismatch
## Model validation for 35 tokens : 100.0% similarity, with 35 matched token, and 0 token mismatch
## Model validation for 40 tokens : 100.0% similarity, with 40 matched token, and 0 token mismatch
## Model validation for 45 tokens : 100.0% similari

## Tune 6 : Ramping up the ctx size (8192), memory training

- Tune 6: Large ctx size (8192), Scaling up!

In [68]:
%%script bash

########################################
# Generate the required jsonl dataset
########################################

# Reset the dataset dir
mkdir -p ../dataset
rm -rf ../dataset/*.jsonl

# Generate the various datasets
echo "## Generating word reptition dataset ##"

#
# We reduce the training set for < 50 words - and shift the focus upwards
# (aka 50-100 token * 2 : ~100 - 250 token ctx len)
#
python ../memory_script/gen_limited_prompt_completion_jsonl.py ../dataset/word-2-count.jsonl 2 50 &
for i in {5..1000..5} 
do
    python ../memory_script/gen_limited_prompt_completion_jsonl.py ../dataset/gen-word-$i-count.jsonl $i 50 & 
    python ../memory_script/shuffle_limited_prompt_completion_jsonl.py ../dataset/shuffle-word-$i-count.jsonl $i 1 & 
done

#
# Ramping up the 50+ - 4200 words dataset
# 
for i in {1100..8000..100} 
do
    python ../memory_script/gen_limited_prompt_completion_jsonl.py ../dataset/gen-word-$i-count.jsonl $i 2000 & 
    python ../memory_script/shuffle_limited_prompt_completion_jsonl.py ../dataset/shuffle-word-$i-count.jsonl $i 20 & 
done

wait
echo "## Done ##"

ls -lh ../dataset/

## Generating word reptition dataset ##
Generated JSONL file with - 2 max words, 50 samples - at ../dataset/word-2-count.jsonl
Generated a single JSONL file with 134 samples (1 token repeat) - 20 max words - at ../dataset/shuffle-word-20-count.jsonl
Generated JSONL file with - 10 max words, 50 samples - at ../dataset/gen-word-10-count.jsonl
Generated JSONL file with - 25 max words, 50 samples - at ../dataset/gen-word-25-count.jsonl
Generated JSONL file with - 55 max words, 50 samples - at ../dataset/gen-word-55-count.jsonl
Generated JSONL file with - 35 max words, 50 samples - at ../dataset/gen-word-35-count.jsonl
Generated JSONL file with - 50 max words, 50 samples - at ../dataset/gen-word-50-count.jsonl
Generated JSONL file with - 40 max words, 50 samples - at ../dataset/gen-word-40-count.jsonl
Generated JSONL file with - 20 max words, 50 samples - at ../dataset/gen-word-20-count.jsonl
Generated a single JSONL file with 34 samples (1 token repeat) - 75 max words - at ../dataset/shuff

In [69]:
# Start the finetune model training
!cd "{TRAINER_DIR}" && \
    export WANDB_MODE="{WANDB_MODE}" && \
    export RWKV_WAVENET_LAYERS="{RWKV_WAVENET_LAYERS}" && \
    python lightning_trainer.py fit \
        -c "{NOTEBOOK_DIR}/v5base-mem-template.yaml" \
        --trainer.logger.init_args.name="{WANDB_PREFIX} - Mem-Tune ctx-8k (train-ctx=4k, {DEEPSPEED_STRAT})" \
        --trainer.strategy="{DEEPSPEED_STRAT}" \
        --trainer.devices="{GPU_DEVICES}"  \
        --trainer.callbacks.init_args.dirpath="../checkpoint/{FILENAME_PREFIX}-mem-ctx-8k/" \
        --model.lr_init=3e-4 \
        --model.lr_final=1e-4 \
        --data.max_token_size=8192 \
        --model.ctx_len=4096 \
        --model.bptt_learning_range=2 \
        --model.load_model="../model/{FILENAME_PREFIX}-mem-ctx-4k.pth"

Setting ds_accelerator to cuda (auto detect)
[RWKV.model] Running RWKV model using 'torch-jit' with torch '2.0.1+cu118'
/usr/local/lib/python3.11/dist-packages/lightning/pytorch/cli.py:484: UserWarning: LightningCLI's args parameter is intended to run from within Python like if it were from the command line. To prevent mistakes it is not recommended to provide both args and command line arguments, got: sys.argv[1:]=['fit', '-c', '/root/rwkv-x-playground/notebook/experiment/rwkv-x-exp/v5-slim-memory/v5base-mem-template.yaml', '--trainer.logger.init_args.name=v5-L6-D4096-E0.1 - Mem-Tune ctx-8k (train-ctx=4k, deepspeed_stage_1)', '--trainer.strategy=deepspeed_stage_1', '--trainer.devices=auto', '--trainer.callbacks.init_args.dirpath=../checkpoint/v5-L6-D4096-E0_1-mem-ctx-8k/', '--model.lr_init=3e-4', '--model.lr_final=1e-4', '--data.max_token_size=8192', '--model.ctx_len=4096', '--model.bptt_learning_range=2', '--model.load_model=../model/v5-L6-D4096-E0_1-mem-ctx-4k.pth'], args=['fit', '-

In [74]:
# Lets export the model from the checkpoint
!cd "{TRAINER_DIR}" && \
    python export_checkpoint.py \
        "../checkpoint/{FILENAME_PREFIX}-mem-ctx-8k/last.ckpt" \
        "../model/{FILENAME_PREFIX}-mem-ctx-8k.pth" "bf16"
!cd "{TRAINER_DIR}" && ls -alh "../model/{FILENAME_PREFIX}-mem-ctx-8k.pth"

Setting ds_accelerator to cuda (auto detect)
Processing zero checkpoint '../checkpoint/v5-L6-D4096-E0_1-mem-ctx-8k/last.ckpt/checkpoint'
Detected checkpoint of type zero stage ZeroStageEnum.optimizer_states, world_size: 8
Parsing checkpoint created by deepspeed==0.9.3
Reconstructed fp32 state dict with 126 params 1720779520 elements
Saving bf16 state dict to ../model/v5-L6-D4096-E0_1-mem-ctx-8k.pth
-rw-r--r-- 1 root root 3.3G Aug 20 17:13 ../model/v5-L6-D4096-E0_1-mem-ctx-8k.pth


In [71]:
# Lets do a quick memory test
!export RWKV_WAVENET_LAYERS="{RWKV_WAVENET_LAYERS}" && \
        python3 ../memory_script/eval_v5_memory_guided.py "{PROJECT_DIR}/model/{FILENAME_PREFIX}-mem-ctx-8k.pth"

Setting ds_accelerator to cuda (auto detect)
[RWKV.model] Running RWKV model using 'torch-jit' with torch '2.0.1+cu118'
###
### Model validation start ###
###
## Model validation for 5 tokens : 100.0% similarity, with 5 matched token, and 0 token mismatch
## Model validation for 10 tokens : 100.0% similarity, with 10 matched token, and 0 token mismatch
## Model validation for 15 tokens : 100.0% similarity, with 15 matched token, and 0 token mismatch
## Model validation for 20 tokens : 100.0% similarity, with 20 matched token, and 0 token mismatch
## Model validation for 25 tokens : 100.0% similarity, with 25 matched token, and 0 token mismatch
## Model validation for 30 tokens : 100.0% similarity, with 30 matched token, and 0 token mismatch
## Model validation for 35 tokens : 100.0% similarity, with 35 matched token, and 0 token mismatch
## Model validation for 40 tokens : 100.0% similarity, with 40 matched token, and 0 token mismatch
## Model validation for 45 tokens : 100.0% similari

In [72]:
!export RWKV_WAVENET_LAYERS="{RWKV_WAVENET_LAYERS}" && \
        python3 ../memory_script/eval_v5_memory_guided.py "{PROJECT_DIR}/model/{FILENAME_PREFIX}-mem-ctx-8k.pth" "none" 1000 4000

Setting ds_accelerator to cuda (auto detect)
[RWKV.model] Running RWKV model using 'torch-jit' with torch '2.0.1+cu118'
###
### Model validation start ###
###
## Model validation for 1000 tokens : 83.6% similarity, with 836 matched token, and 164 token mismatch
## Model validation for 1050 tokens : 81.71428571428572% similarity, with 858 matched token, and 192 token mismatch
## Model validation for 1100 tokens : 80.0909090909091% similarity, with 881 matched token, and 219 token mismatch
## Model validation for 1150 tokens : 78.34782608695652% similarity, with 901 matched token, and 249 token mismatch
## Model validation for 1200 tokens : 77.08333333333334% similarity, with 925 matched token, and 275 token mismatch
## Model validation for 1250 tokens : 74.56% similarity, with 932 matched token, and 318 token mismatch
## Model validation for 1300 tokens : 73.07692307692307% similarity, with 950 matched token, and 350 token mismatch
## Model validation for 1350 tokens : 72.5925925925926%

# Side track - positional loss expeirment

In [73]:
# Start the finetune model training
!cd "{TRAINER_DIR}" && \
    export WANDB_MODE="{WANDB_MODE}" && \
    export RWKV_WAVENET_LAYERS="{RWKV_WAVENET_LAYERS}" && \
    python lightning_trainer.py fit \
        -c "{NOTEBOOK_DIR}/v5base-mem-template.yaml" \
        --trainer.logger.init_args.name="{WANDB_PREFIX} - Mem-Tune ctx-8k (pos_loss_bias=0.01, train-ctx=4k, {DEEPSPEED_STRAT})" \
        --trainer.strategy="{DEEPSPEED_STRAT}" \
        --trainer.devices="{GPU_DEVICES}"  \
        --trainer.callbacks.init_args.dirpath="../checkpoint/{FILENAME_PREFIX}-mem-ctx-8k-p0_01/" \
        --model.lr_init=3e-4 \
        --model.lr_final=1e-4 \
        --data.max_token_size=8192 \
        --model.position_loss_bias=0.01 \
        --model.ctx_len=4096 \
        --model.bptt_learning_range=2 \
        --model.load_model="../model/{FILENAME_PREFIX}-mem-ctx-4k.pth"

Setting ds_accelerator to cuda (auto detect)
[RWKV.model] Running RWKV model using 'torch-jit' with torch '2.0.1+cu118'
/usr/local/lib/python3.11/dist-packages/lightning/pytorch/cli.py:484: UserWarning: LightningCLI's args parameter is intended to run from within Python like if it were from the command line. To prevent mistakes it is not recommended to provide both args and command line arguments, got: sys.argv[1:]=['fit', '-c', '/root/rwkv-x-playground/notebook/experiment/rwkv-x-exp/v5-slim-memory/v5base-mem-template.yaml', '--trainer.logger.init_args.name=v5-L6-D4096-E0.1 - Mem-Tune ctx-8k (pos_loss_bias=0.01, train-ctx=4k, deepspeed_stage_1)', '--trainer.strategy=deepspeed_stage_1', '--trainer.devices=auto', '--trainer.callbacks.init_args.dirpath=../checkpoint/v5-L6-D4096-E0_1-mem-ctx-8k/', '--model.lr_init=3e-4', '--model.lr_final=1e-4', '--data.max_token_size=8192', '--model.position_loss_bias=0.01', '--model.ctx_len=4096', '--model.bptt_learning_range=2', '--model.load_model=../m

In [75]:
# Lets export the model from the checkpoint
!cd "{TRAINER_DIR}" && \
    python export_checkpoint.py \
        "../checkpoint/{FILENAME_PREFIX}-mem-ctx-8k/last-v1.ckpt" \
        "../model/{FILENAME_PREFIX}-mem-ctx-8k-p0_01.pth" "bf16"
!cd "{TRAINER_DIR}" && ls -alh "../model/{FILENAME_PREFIX}-mem-ctx-8k-p0_01.pth"

Setting ds_accelerator to cuda (auto detect)
Processing zero checkpoint '../checkpoint/v5-L6-D4096-E0_1-mem-ctx-8k/last-v1.ckpt/checkpoint'
Detected checkpoint of type zero stage ZeroStageEnum.optimizer_states, world_size: 8
Parsing checkpoint created by deepspeed==0.9.3
Reconstructed fp32 state dict with 126 params 1720779520 elements
Saving bf16 state dict to ../model/v5-L6-D4096-E0_1-mem-ctx-8k-p0_01.pth
-rw-r--r-- 1 root root 3.3G Aug 20 17:14 ../model/v5-L6-D4096-E0_1-mem-ctx-8k-p0_01.pth


In [76]:
# Lets do a quick memory test
!export RWKV_WAVENET_LAYERS="{RWKV_WAVENET_LAYERS}" && \
        python3 ../memory_script/eval_v5_memory_guided.py "{PROJECT_DIR}/model/{FILENAME_PREFIX}-mem-ctx-8k-p0_01.pth"

Setting ds_accelerator to cuda (auto detect)
[RWKV.model] Running RWKV model using 'torch-jit' with torch '2.0.1+cu118'
###
### Model validation start ###
###
## Model validation for 5 tokens : 100.0% similarity, with 5 matched token, and 0 token mismatch
## Model validation for 10 tokens : 100.0% similarity, with 10 matched token, and 0 token mismatch
## Model validation for 15 tokens : 100.0% similarity, with 15 matched token, and 0 token mismatch
## Model validation for 20 tokens : 100.0% similarity, with 20 matched token, and 0 token mismatch
## Model validation for 25 tokens : 100.0% similarity, with 25 matched token, and 0 token mismatch
## Model validation for 30 tokens : 100.0% similarity, with 30 matched token, and 0 token mismatch
## Model validation for 35 tokens : 100.0% similarity, with 35 matched token, and 0 token mismatch
## Model validation for 40 tokens : 100.0% similarity, with 40 matched token, and 0 token mismatch
## Model validation for 45 tokens : 100.0% similari

In [77]:
!export RWKV_WAVENET_LAYERS="{RWKV_WAVENET_LAYERS}" && \
        python3 ../memory_script/eval_v5_memory_guided.py "{PROJECT_DIR}/model/{FILENAME_PREFIX}-mem-ctx-8k-p0_01.pth" "none" 1000 4000

Setting ds_accelerator to cuda (auto detect)
[RWKV.model] Running RWKV model using 'torch-jit' with torch '2.0.1+cu118'
###
### Model validation start ###
###
## Model validation for 1000 tokens : 71.7% similarity, with 717 matched token, and 283 token mismatch
## Model validation for 1050 tokens : 70.19047619047619% similarity, with 737 matched token, and 313 token mismatch
## Model validation for 1100 tokens : 68.45454545454545% similarity, with 753 matched token, and 347 token mismatch
## Model validation for 1150 tokens : 65.39130434782608% similarity, with 752 matched token, and 398 token mismatch
## Model validation for 1200 tokens : 64.41666666666667% similarity, with 773 matched token, and 427 token mismatch
## Model validation for 1250 tokens : 63.519999999999996% similarity, with 794 matched token, and 456 token mismatch
## Model validation for 1300 tokens : 62.69230769230769% similarity, with 815 matched token, and 485 token mismatch
## Model validation for 1350 tokens : 60.